Shortcuts:
    - Move cell down: .
    - Move cell up: /

# Anomaly detection using Facebook Prophet:

*This notebook is based on the following [tutorial](https://medium.com/analytics-vidhya/time-series-forecast-anomaly-detection-with-facebook-prophet-558136be4b8d) written by Paul Lo. It makes use of the open-source project [Prophet](https://facebook.github.io/prophet/), a forecasting procedure implemented in R and Python, based on the paper of [Taylor and Letham, 2017](https://peerj.com/preprints/3190/).*

**Goal of the script:**

Here, we aim to the stress-related detect changes related to stress and is based on the analysis of the WESAD dataset.

**Motivations to use a forecasting method to detect activity:**

Previous works demonstrated the ability to related self-labeled stress status to sensor data acquired by wearable sensors.
Here we try a different approach assuming that physiological rythms are altered by stress. We are investigating if a time series forecasting method coupled with anomaly detection provides a more sensitive methods to detect stress-related changes.

**Data format**

The reader may read the
- [UCI website](https://archive.ics.uci.edu/ml/datasets/WESAD+%28Wearable+Stress+and+Affect+Detection%29) (check the website shown below) to download the WESAD dataset 
- [wesad_readme file](wesad_readme.pdf) and [wesad poster](WESAD poster.pdf), both located together with the WESAD dataset

> Questions:
> Contact Guillaume Azarias at guillaume.azarias@hotmail.com

## Import the relevant library

In [1]:
import os
import pickle
import numpy as np
import seaborn as sns
sns.set()
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import timedelta
import qgrid

# Note that the interactive plot may not work in Jupyter lab, but only in Jupyter Notebook (conflict of javascripts)
%matplotlib widget 

In [2]:
import fbprophet
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation, performance_metrics
from fbprophet.plot import plot_cross_validation_metric

In [3]:
fbprophet.__version__

'0.6'

In [4]:
from sklearn.model_selection import ParameterGrid
import itertools
from random import sample

In [5]:
# Import the functions from the helper.py
from helper import load_ds, df_dev_formater, find_index, df_generator, prophet_fit, prophet_plot, get_outliers, prophet, GridSearch_Prophet

## Read the WESAD data

The loading script was modified from [a repository from aganjag](https://github.com/jaganjag/stress_affect_detection/blob/master/prototype.ipynb)

The dimensions of the dataset depend on both the device and parameters:

|     Device     | Location|Parameter|Acq. frequency|Number of dimensions|Data points (S5)| Duration (S5)|
|:---------------|:-------:|:-------:|:------------:|:------------------:|:--------------:|:------------:|
|**RespiBAN Pro**|chest    | ACC     |700Hz         |**3**               |4496100         |6'423sec      |
|                |         | ECG     |"             |1                   |                |              |
|                |         | EDA     |"             |1                   |                |              |
|                |         | EMG     |"             |1                   |                |              |
|                |         | RESP    |"             |1                   |                |              |
|                |         | TEMP    |"             |1                   |                |              |
|                |         |         |              |                    |                |              |
|**Empatica E4** |wrist    | ACC     |32Hz          |**3**               |200256          |6'258sec      |
|                |         | BVP     |64Hz          |1                   |400512          |              |
|                |         | EDA     |4Hz           |1                   |25032           |              |
|                |         | TEMP    |4Hz           |1                   |25032           |              |

*Note that ACC is a matrix of 3 dimensions for the 3 spatial dimensions*

*'ECG', 'EDA', 'EMG', 'Resp', 'Temp' have each 1 dimension.*

In [6]:
# obj_data[subject] = read_data_one_subject(data_set_path, subject)
class read_data_of_one_subject:
    """Read data from WESAD dataset"""
    def __init__(self, path, subject):
        self.keys = ['label', 'subject', 'signal']
        self.signal_keys = ['wrist', 'chest']
        self.chest_sensor_keys = ['ACC', 'ECG', 'EDA', 'EMG', 'Resp', 'Temp']
        self.wrist_sensor_keys = ['ACC', 'BVP', 'EDA', 'TEMP']
        os.chdir(path) # Change the current working directory to path
        os.chdir(subject) # Change the current working directory to path. Why not using data_set_path ?
        with open(subject + '.pkl', 'rb') as file: # with will automatically close the file after the nested block of code
            data = pickle.load(file, encoding='latin1')
        self.data = data

    def get_labels(self):
        return self.data[self.keys[0]]

    def get_wrist_data(self):
        """"""
        #label = self.data[self.keys[0]]
        assert subject == self.data[self.keys[1]], 'WARNING: Mixing up the data from different persons'
        signal = self.data[self.keys[2]]
        wrist_data = signal[self.signal_keys[0]]
        #wrist_ACC = wrist_data[self.wrist_sensor_keys[0]]
        #wrist_ECG = wrist_data[self.wrist_sensor_keys[1]]
        return wrist_data

    def get_chest_data(self):
        """"""
        assert subject == self.data[self.keys[1]], 'WARNING: Mixing up the data from different persons'
        signal = self.data[self.keys[2]]
        chest_data = signal[self.signal_keys[1]]
        return chest_data

In [7]:
data_set_path = "../../Data/WESAD"
subject = 'S5'

In [8]:
obj_data = {}
obj_data[subject] = read_data_of_one_subject(data_set_path, subject)

*Workplan:*

**A) Exploratory data analysis**

    1) Discard for now the ACC data. Preliminary results on other parameters may guide the ways to investigate the accelerometer data
    2) Get the study protocol
    3) Use rolling.mean() to synchronise the data at the same frequency
    4) Synchronise data
    5) Include label data if possible
    6) Plot data

**B) Perform time series forecasting**

    1) ADCF test
    2) Prophet
    3) ARIMA

### Get the study protocol
*From the wesad_readme.pdf:*

The order of the different conditions is defined on the second line in SX_quest.csv. Please refer to [1] for further details on each of the conditions (see Section 3.3 there). Please ignore the elements “bRead”, “fRead”, and “sRead”: these are not relevant for this dataset.
The time interval of each condition is defined as start and end time, see the lines 3 and 4 in SX_quest.csv. Time is given in the format [minutes.seconds]. Time is counted from the start of the RespiBAN device’s start of recording.

In [9]:
print(os.getcwd())

/Users/guillaume/Documents/Projects/Data/WESAD/S5


In [10]:
SX_quest_filename = os.getcwd() + '/' + subject + '_quest.csv'
print(SX_quest_filename)
# bp_data = pd.read_csv("/Users/guillaume/Documents/Projects/Data/WESAD/S2/S2_quest.csv", header=1, delimiter=';')
study_protocol_raw = pd.read_csv(SX_quest_filename, delimiter=';')
study_protocol_raw.head()

/Users/guillaume/Documents/Projects/Data/WESAD/S5/S5_quest.csv


,# Subj,S5,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26
0,# ORDER,Base,Fun,Medi 1,TSST,Medi 2,bRead,fRead,sRead,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,# START,5.37,32,45.43,61,92.15,26.15,41.15,75.28,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,# END,25.55,38.34,52.4,72.05,99.12,27.47,42.45,76.32,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,# PANAS,1,1,4,3,1,3,1,1,1.0,...,4.0,4.0,1.0,1.0,2.0,1.0,3.0,1.0,NaN,NaN


In [11]:
# Create a table with the interval of every steps
study_protocol = study_protocol_raw.iloc[1:3, 1:6]
study_protocol = study_protocol.transpose().astype(float)
study_protocol.columns = ['start', 'end']
study_protocol['task'] = study_protocol_raw.iloc[0, 1:6].transpose()
study_protocol = study_protocol.reset_index(drop=True)
study_protocol
# study_protocol.dtypes

,start,end,task
0,5.37,25.55,Base
1,32.00,38.34,Fun
2,45.43,52.40,Medi 1
3,61.00,72.05,TSST
4,92.15,99.12,Medi 2


In [55]:
# Create a dataframe with the time formatted as datetime
# Note that the frequency chosen was 4Hz to match the lowest frequency of acquisition (250ms)
total_duration = study_protocol.end.max()
data = pd.DataFrame()
begin_df = datetime.datetime(2020, 1, 1) # For reading convenience
print(begin_df)
end_df = begin_df + timedelta(minutes=int(total_duration)) + timedelta(seconds=total_duration-int(total_duration))
print(end_df)
data['time'] = pd.date_range(start=begin_df, end=end_df, freq='250L').to_pydatetime().tolist()
data['task'] = np.nan
data

2020-01-01 00:00:00
2020-01-01 01:39:00.120000


,time,task
0,2020-01-01 00:00:00.000,NaN
1,2020-01-01 00:00:00.250,NaN
2,2020-01-01 00:00:00.500,NaN
3,2020-01-01 00:00:00.750,NaN
4,2020-01-01 00:00:01.000,NaN
...,...,...
23756,2020-01-01 01:38:59.000,NaN
23757,2020-01-01 01:38:59.250,NaN
23758,2020-01-01 01:38:59.500,NaN
23759,2020-01-01 01:38:59.750,NaN


In [54]:
row = 0
# Annotation of the data with the task
print(begin_df)
begin_state = study_protocol.iloc[row, 0]
begin = begin_df + timedelta(minutes=int(begin_state)) + timedelta(seconds=begin_state-int(begin_state))
print(begin)
end_state = study_protocol.iloc[row, 1]
end = begin_df + timedelta(minutes=int(end_state)) + timedelta(seconds=end_state-int(end_state))
print(end)

2020-01-01 00:00:00
2020-01-01 00:05:00.370000
2020-01-01 00:25:00.550000


#TODO: Iterate data annotation according to the study protocol

In [59]:
data.loc[(data['time'] >= begin) & (data['time'] <= end), 'task'] = study_protocol.iloc[row, 2]
qgrid_widget = qgrid.show_grid(data, show_toolbar=True)
qgrid.show_grid(data)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

### Get the wrist data

In [ ]:
bp_data = pd.read_csv("/Users/guillaume/Documents/Projects/Data/WESAD/S3/S3_quest.csv", header=1, delimiter=';')
bp_data.head()

In [ ]:
bp_data = pd.read_csv("/Users/guillaume/Documents/Projects/Data/WESAD/S4/S4_quest.csv", header=1, delimiter=';')
bp_data.head()

In [ ]:
bp_data = pd.read_csv("/Users/guillaume/Documents/Projects/Data/WESAD/S5/S5_quest.csv", delimiter=';')
# qgrid_widget = qgrid.show_grid(bp_data, show_toolbar=True)
# qgrishow_gridrid(bp_data)
bp_data.head(10)

In [ ]:
bp_data = pd.read_csv("/Users/guillaume/Documents/Projects/Data/WESAD/S5/S5_quest.csv", header=1, delimiter=';')
# qgrid_widget = qgrid.show_grid(bp_data, show_toolbar=True)
# qgrishow_gridrid(bp_data)
bp_data.head()

In [ ]:
bp_data.columns

In [ ]:
6423/60

In [ ]:
bp_data = pd.read_csv("/Users/guillaume/Documents/Projects/Data/WESAD/S6/S6_quest.csv", header=1, delimiter=';')
bp_data.head()

In [ ]:
wrist_data_dict = obj_data[subject].get_wrist_data()
wrist_dict_length = {key: len(value) for key, value in wrist_data_dict.items()}
print(wrist_dict_length)

In [ ]:
# blood volume pulse (BVP, 64Hz)
BVP_data = wrist_data_dict['BVP'].flatten()
# type(BVP_data)
df_BVP = pd.DataFrame(data=BVP_data, columns=['BVP'])
type(df_BVP_raw)

In [ ]:
# Electrodermal activity and Temperature (EDA, TEMP, 4Hz)
wrist_data = np.concatenate((wrist_data_dict['EDA'], wrist_data_dict['TEMP']), axis=1)
# type(wrist_data)
df_EDA_TEMP = pd.DataFrame(data=wrist_data, columns=['EDA', 'TEMP'])
type(df_EDA_TEMP)

In [ ]:
df_BVP = df_BVP.resample(sampling_period_st).pad()

In [ ]:
# Show the data
qgrid_widget = qgrid.show_grid(df_EDA_TEMP, show_toolbar=True)
qgrid.show_grid(df_EDA_TEMP)

### Get the chest data

In [ ]:
chest_data_dict = obj_data[subject].get_chest_data()
chest_dict_length = {key: len(value) for key, value in chest_data_dict.items()}
print(chest_dict_length)

In [ ]:
# Convert the list to an np array
# chest_data = np.concatenate((chest_data_dict['ACC'], chest_data_dict['ECG'], chest_data_dict['EDA'], chest_data_dict['EMG'], chest_data_dict['Resp'], chest_data_dict['Temp']), axis=1)
chest_data = np.concatenate((chest_data_dict['ECG'], chest_data_dict['EDA'], chest_data_dict['EMG'], chest_data_dict['Resp'], chest_data_dict['Temp']), axis=1)
type(chest_data)

In [ ]:
# Convert the np array into a pandas DataFrame
df = pd.DataFrame(data=chest_data, columns=['ECG', 'EDA', 'EMG', 'Resp', 'Temp'])
type(df)

In [ ]:
# Show the data
qgrid_widget = qgrid.show_grid(df, show_toolbar=True)
qgrid.show_grid(df)

In [ ]:
df.shape

### Get the label data

In [ ]:
# No. of Labels ==> 8 ; 0 = not defined / transient, 1 = baseline, 2 = stress, 3 = amusement,
# 4 = meditation, 5/6/7 = should be ignored in this dataset
labels = {}
labels[subject] = obj_data[subject].get_labels()

In [ ]:
import sys
sys.getsizeof(labels)
# len(labels)

In [ ]:
# snippet plot
# Silly example data
bp_x = np.linspace(0, 2*np.pi, num=40, endpoint=True)
bp_y = np.sin(bp_x)

# Make the plot
plt.plot(bp_x, bp_y, linewidth=3, linestyle="--",
         color="blue", label=r"Legend label $\sin(x)$")
plt.xlabel(r"Description of $x$ coordinate (units)")
plt.ylabel(r"Description of $y$ coordinate (units)")
plt.title(r"Title here (remove for papers)")
plt.xlim(0, 2*np.pi)
plt.ylim(-1.1, 1.1)
plt.legend(loc="lower left")
plt.show()

In [ ]:
from __future__ import print_function, division
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline



In [ ]:
def extract_one(chest_data_dict, idx, l_condition=0):
    ecg_data = chest_data_dict["ECG"][idx].flatten()
    ecg_features = extract_mean_std_features(ecg_data, label=l_condition)
    #print(ecg_features.shape)

    eda_data = chest_data_dict["EDA"][idx].flatten()
    eda_features = extract_mean_std_features(eda_data, label=l_condition)
    #print(eda_features.shape)

    emg_data = chest_data_dict["EMG"][idx].flatten()
    emg_features = extract_mean_std_features(emg_data, label=l_condition)
    #print(emg_features.shape)

    temp_data = chest_data_dict["Temp"][idx].flatten()
    temp_features = extract_mean_std_features(temp_data, label=l_condition)
    #print(temp_features.shape)

    baseline_data = np.hstack((eda_features, temp_features, ecg_features, emg_features))
    #print(len(baseline_data))
    label_array = np.full(len(baseline_data), l_condition)
    #print(label_array.shape)
    #print(baseline_data.shape)
    baseline_data = np.column_stack((baseline_data, label_array))
    #print(baseline_data.shape)
    return baseline_data

In [ ]:
def execute():
#     data_set_path = "/media/jac/New Volume/Datasets/WESAD"
    data_set_path = "../../../Data/WESAD"
    file_path = "ecg.txt"
    subject = 'S3' # Why defining subject here since it is defined 6 lines later in a loop ?
    obj_data = {}
    labels = {}
    all_data = {}
    subs = [2, 3, 4, 5, 6]
    for i in subs:
        subject = 'S' + str(i)
        print("Reading data", subject)
        obj_data[subject] = read_data_one_subject(data_set_path, subject)
        labels[subject] = obj_data[subject].get_labels()

        wrist_data_dict = obj_data[subject].get_wrist_data()
        wrist_dict_length = {key: len(value) for key, value in wrist_data_dict.items()}

        chest_data_dict = obj_data[subject].get_chest_data()
        chest_dict_length = {key: len(value) for key, value in chest_data_dict.items()}
        print(chest_dict_length)
        chest_data = np.concatenate((chest_data_dict['ACC'], chest_data_dict['ECG'], chest_data_dict['EDA'],
                                     chest_data_dict['EMG'], chest_data_dict['Resp'], chest_data_dict['Temp']), axis=1)
        # Get labels


        # 'ACC' : 3, 'ECG' 1: , 'EDA' : 1, 'EMG': 1, 'RESP': 1, 'Temp': 1  ===> Total dimensions : 8
        # No. of Labels ==> 8 ; 0 = not defined / transient, 1 = baseline, 2 = stress, 3 = amusement,
        # 4 = meditation, 5/6/7 = should be ignored in this dataset

        # Do for each subject
        baseline = np.asarray([idx for idx, val in enumerate(labels[subject]) if val == 1])
        # print("Baseline:", chest_data_dict['ECG'][baseline].shape)
        # print(baseline.shape)

        stress = np.asarray([idx for idx, val in enumerate(labels[subject]) if val == 2])
        # print(stress.shape)

        amusement = np.asarray([idx for idx, val in enumerate(labels[subject]) if val == 3])
        # print(amusement.shape)

        baseline_data = extract_one(chest_data_dict, baseline, l_condition=1)
        stress_data = extract_one(chest_data_dict, stress, l_condition=2)
        amusement_data = extract_one(chest_data_dict, amusement, l_condition=3)

        full_data = np.vstack((baseline_data, stress_data, amusement_data))
        print("One subject data", full_data.shape)
        all_data[subject] = full_data

    i = 0
    for k, v in all_data.items():
        if i == 0:
            data = all_data[k]
            i += 1
        print(all_data[k].shape)
        data = np.vstack((data, all_data[k]))

    print(data.shape)
    return data

In [ ]:
# """
ecg, eda = chest_data_dict['ECG'], chest_data_dict['EDA']
x = [i for i in range(len(baseline))]
for one in baseline:
    x = [i for i in range(99)]
    plt.plot(x, ecg[one:100])
    break
# """

x = [i for i in range(10000)]
plt.plot(x, chest_data_dict['ECG'][:10000])
plt.show()

# BASELINE

                                   [ecg_features[k] for k in ecg_features.keys()])

ecg = nk.ecg_process(ecg=ecg_data, rsp=chest_data_dict['Resp'][baseline].flatten(), sampling_rate=700)
print(os.getcwd())

# """
recur_print
print(type(ecg))
print(ecg.keys())
for k in ecg.keys():
    print(k)
    for i in ecg[k].keys():
        print(i)
    
resp = nk.eda_process(eda=chest_data_dict['EDA'][baseline].flatten(), sampling_rate=700)
resp = nk.rsp_process(chest_data_dict['Resp'][baseline].flatten(), sampling_rate=700)
for k in resp.keys():
    print(k)
    for i in resp[k].keys():
        print(i)
    
# For baseline, compute mean, std, for each 700 samples. (1 second values)
file_path = os.getcwd()
with open(file_path, "w") as file:
    #file.write(str(ecg['df']))
    file.write(str(ecg['ECG']['HRV']['RR_Intervals']))
    file.write("...")
    file.write(str(ecg['RSP']))
    #file.write("RESP................")
    #file.write(str(resp['RSP']))
    #file.write(str(resp['df']))
    #print(type(ecg['ECG']['HRV']['RR_Intervals']))
    #file.write(str(ecg['ECG']['Cardiac_Cycles']))
    #print(type(ecg['ECG']['Cardiac_Cycles']))
    #file.write(ecg['ECG']['Cardiac_Cycles'].to_csv())
    
    
# Plot the processed dataframe, normalizing all variables for viewing purpose
# """
# """
bio = nk.bio_process(ecg=chest_data_dict["ECG"][baseline].flatten(), rsp=chest_data_dict['Resp'][baseline].flatten(), eda=chest_data_dict["EDA"][baseline].flatten(), sampling_rate=700)
nk.z_score(bio["df"]).plot()
print(bio["ECG"].keys())
print(bio["EDA"].keys())
print(bio["RSP"].keys())
#ECG
print(bio["ECG"]["HRV"])
print(bio["ECG"]["R_Peaks"])
#EDA
print(bio["EDA"]["SCR_Peaks_Amplitudes"])
print(bio["EDA"]["SCR_Onsets"])
#RSP
print(bio["RSP"]["Cycles_Onsets"])
print(bio["RSP"]["Cycles_Length"])
# """
print("Read data file")
#Flow: Read data for all subjects -> Extract features (Preprocessing) -> Train the model

In [ ]:
data_set_path = "../../../Data/WESAD"
subject = 'S4'

In [ ]:
obj_data = {}
obj_data[subject] = read_data_of_one_subject(data_set_path, subject)

In [ ]:
chest_data_dict = obj_data[subject].get_chest_data()
chest_dict_length = {key: len(value) for key, value in chest_data_dict.items()}
print(chest_dict_length)

In [ ]:
# Get labels
labels = obj_data[subject].get_labels()
baseline = np.asarray([idx for idx,val in enumerate(labels) if val == 1])
#print(baseline)

print("Baseline:", chest_data_dict['ECG'][baseline].shape)

In [ ]:
labels.shape

In [ ]:
from sklearn.externals import joblib

In [ ]:
bio = nk.bio_process(ecg=chest_data_dict["ECG"][baseline].flatten(), rsp=chest_data_dict['Resp'][baseline].flatten(), eda=chest_data_dict["EDA"][baseline].flatten(), sampling_rate=700)
nk.z_score(bio["df"]).plot()
"""print(bio["ECG"].keys())
print(bio["EDA"].keys())
print(bio["RSP"].keys())

#ECG
print(bio["ECG"]["HRV"])
print(bio["ECG"]["R_Peaks"])

#EDA
print(bio["EDA"]["SCR_Peaks_Amplitudes"])
print(bio["EDA"]["SCR_Onsets"])

#RSP
print(bio["RSP"]["Cycles_Onsets"])
print(bio["RSP"]["Cycles_Length"])
"""

### Try to display the dataframe with qgrid:

Check the [quantopian link](https://github.com/quantopian/qgrid).

```python
import qgrid
qgrid_widget = qgrid_widget.show_grid(df, show_toolbar=True)
qgrid_widget
```

# Descriptive statistics in Time Series Modelling
https://towardsdatascience.com/descriptive-statistics-in-time-series-modelling-db6ec569c0b8

Stationarity
A time series is said to be stationary if it doesn’t increase or decrease with time linearly or exponentially(no trends), and if it doesn’t show any kind of repeating patterns(no seasonality). Mathematically, this is described as having constant mean and constant variance over time. Along, with variance, the autocovariance should also not be a function of time. If you have forgotten what mean and variance are: mean is the average of the data and variance is the average squared distance from the mean.

Sometimes, it’s even difficult to interpret the rolling mean visually so we take the help of statistical tests to identify this, one such being Augmented Dickey Fuller Test. ADCF Test is implemented using statsmodels in python which performs a classic null hypothesis test and returns a p-value.
Interpretation of null hypothesis test: If p-value is less than 0.05 (p-value: low), we reject the null hypothesis and assume that the data is stationary. But if the p-value is more than 0.05 (p-value: high), then we fail to reject the null hypothesis and determine the data to be non-stationary.

## 1.2 Random Forest Classifier (from jaganjag Github)

*Not sure if it would be relevant but keeping the code for completeness of the repo*

In [ ]:
from read_data import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture

if __name__ == '__main__':
    data = execute()
    print(data.shape)
    X = data[:, :16]  # 16 features
    y = data[:, 16]
    print(X.shape)
    print(y.shape)
    print(y)
    train_features, test_features, train_labels, test_labels = train_test_split(X, y,
                                                                                test_size=0.25)
    print('Training Features Shape:', train_features.shape)
    print('Training Labels Shape:', train_labels.shape)
    print('Testing Features Shape:', test_features.shape)
    print('Testing Labels Shape:', test_labels.shape)

    clf = RandomForestClassifier(n_estimators=100, max_depth=5, oob_score=True)
    clf.fit(X, y)
    print(clf.feature_importances_)
    # print(clf.oob_decision_function_)
    print(clf.oob_score_)
    predictions = clf.predict(test_features)
    errors = abs(predictions - test_labels)
    print("M A E: ", np.mean(errors))
    print(np.count_nonzero(errors), len(test_labels))
    print("Accuracy:", np.count_nonzero(errors)/len(test_labels))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture

import numpy as np

X, y = make_classification(n_samples=10000, n_features=6,
                            n_informative=3, n_redundant=0,
                            random_state=0, shuffle=True)

print(X.shape)  # 10000x6
print(y.shape)  # 10000

# TODO: Feature extraction using sliding window

train_features, test_features, train_labels, test_labels = train_test_split(X, y,
                                                                            test_size=0.25, random_state=42)
# TODO: K-fold cross validation

print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

clf = RandomForestClassifier(n_estimators=100, max_depth=3, oob_score=True
                             )

clf.fit(X, y)

print(clf.feature_importances_)
#print(clf.oob_decision_function_)
print(clf.oob_score_)

predictions = clf.predict(test_features)
errors = abs(predictions - test_labels)
print("M A E: ", round(np.mean(errors), 2))


# Visualization
feature_list = [1, 2, 3, 4, 5, 6]
from sklearn.tree import export_graphviz
import pydot
# Pull out one tree from the forest
tree = clf.estimators_[5]
# Export the image to a dot file
export_graphviz(tree, out_file='tree.dot', feature_names=feature_list, rounded=True, precision=1)
# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')
# Write graph to a png file
#graph.write_png('tree_.png')

# TODO: Confusion matrix, Accuracy


# GMM

gmm = GaussianMixture(n_components=3, covariance_type='full')
gmm.fit(X, y)

## Function to downsample the dataset, run a GridSearch, sort the best model according to the mean average percentage error

* Downsampling of dataset: Pick 10 days in a device-specific dataset and will run the GridSearch. Allow to run the algorithm on all device-specific dataframes.
* GridSearch trying Prophet with different training periods (8, 10 or 12 training days). This was the most critical parameter affecting the mean average percentage error (mape).
* Sort the Prophet model according to the mape. Save the best model with graph and a dataframe containing the prediction and actual data.

In [ ]:
n_samples = 10 # Limit to 10 predictions per device.
pred_duration = 12 # 12-day prediction

for dev_nb in range(1,52):
    device_nb = str('{:02d}'.format(dev_nb))
    # Load the device-specific dataframe.
    assert isinstance(device_nb, str) and len(device_nb)==2 and sum(d.isdigit() for d in device_nb)==2, 'WARNING: device_nb must be a string of 2-digits!'
    assert int(device_nb)>=1 and int(device_nb)<=51, 'This device does not belong to the dataframe'
    device, df_dev = load_ds(device_nb)
    # Convert the variable device from a np.array to a string
    regex = re.compile('[^A-Za-z0-9]')
    device = regex.sub('', str(device))
    
    # Create a dataframe with the dates to use
    dates = pd.DataFrame(columns={'date_minus_12',
                                  'date_minus_10',
                                  'date_minus_8',
                                  'date_predict'})
    dates = dates[['date_minus_12', 'date_minus_10', 'date_minus_8', 'date_predict']]
    # List of unique dates in the dataframe
    dates['date_minus_12'] = df_dev['ds'].unique().strftime('%Y-%m-%d')
    dates = dates.drop_duplicates(subset=['date_minus_12'])
    dates = dates.reset_index(drop=True)
    # Fill the other columns and drop the 12 last columns
    dates['date_minus_10'] = dates.iloc[2:, 0].reset_index(drop=True)
    dates['date_minus_8'] = dates.iloc[4:, 0].reset_index(drop=True)
    dates['date_predict'] = dates.iloc[12:, 0].reset_index(drop=True)
    dates = dates[:-pred_duration] # Drop the 12 last rows
    
    # Keep only the dates with at least 12 training days
    dates['Do_It'] = 'Do not'
    dates['dm_12_c'] = np.nan
    
    for r in range(dates.shape[0]):
        # Calculate the date_predict - pred_duration
        date_predict = dates.iloc[r, 3]
        date_predict = datetime.strptime(date_predict, "%Y-%m-%d")

        date_minus_12_check = date_predict + timedelta(days=-pred_duration)
        date_minus_12_check = datetime.strftime(date_minus_12_check, "%Y-%m-%d")
        
        # Tag the date_predict that have at least 12 training days
        if date_minus_12_check in dates.date_predict.values or r<=11:
            dates.iloc[r, 4] = 'yes'

    dates = dates[dates.Do_It == 'yes']
    dates.drop(['Do_It', 'dm_12_c'], axis=1)
    
    # Downsampling
    if dates.shape[0]>n_samples:
        dates = dates.sample(n=n_samples, replace=False)

    # GridSearch over the (down-sampled) dataset:
    start_time = time.time()
    mape_table_full = pd.DataFrame()
    
    for r in range(dates.shape[0]):
        # Parameters of the Grid
        prophet_grid = {'df_dev' : [df_dev],
                        'device' : [device],
                        'parameter' : ['co2'],
                        'begin' : dates.iloc[r, :3].tolist(),
                        'end' : [dates.iloc[r, 3]],
                        'sampling_period_min' : [1],
                        'graph' : [1],
                        'predict_day' : [1],
                        'interval_width' : [0.6],
                        'changepoint_prior_scale' : [0.01, 0.005], # list(np.arange(0.01,30,1).tolist()),
                        'daily_fo' : [3],
            #             'holidays_prior_scale' : list((1000,100,10,1,0.1)),
                           }

        # Run GridSearch_Prophet
        mape_table = GridSearch_Prophet(list(ParameterGrid(prophet_grid)), metric='mape')
        mape_table_full = mape_table_full.append(mape_table)

        end_time = time.time()
        dur_min = int((end_time - start_time)/60)
        
        print('Time elapsed: '+ str(dur_min) + " minutes.")

        # Save the best model
        print('Saving the best model')
        
        best_model = {'df_dev' : [df_dev],
                      'device' : [mape_table.iloc[0, 0]],
                      'parameter' : [mape_table.iloc[0, 1]],
                      'begin' : [mape_table.iloc[0, 2]],
                      'end' : [mape_table.iloc[0, 3]],
                      'sampling_period_min' : [mape_table.iloc[0, 4]],
                      'graph' : [1],
                      'predict_day' : [1],
                      'interval_width' : [mape_table.iloc[0, 5]],
                      'changepoint_prior_scale' : [mape_table.iloc[0, 7]], # list(np.arange(0.01,30,1).tolist()),
                      'daily_fo' : [mape_table.iloc[0, 6]],
#                       'holidays_prior_scale' : list((1000,100,10,1,0.1)),
                           }

        # Run GridSearch_Prophet on the best model
        mape_table = GridSearch_Prophet(list(ParameterGrid(best_model)), metric='mape')

        end_time = time.time()
        dur_min = int((end_time - start_time)/60)
        print('Full analysis completed in '+ str(dur_min) + ' minutes.')

    # Save the full table of mape_table
    # Store the complete mape_table if this is the last prediction
    folder_name = '/Users/guillaume/Documents/DS2020/Caru/caru/data/processed/'
    mape_table_name = folder_name + re.sub("[']", '', str(mape_table.iloc[0, 0])) + '_mape_table_full.csv'
    mape_table_full.to_csv(mape_table_name)